# COM6012 - 2017: Practical Quiz 1

## Question 1 [2 marks]

Use the dataset [Census Income Data](./files/CensusIncomeData.csv) to find out how many women have a Masters degree. Provide your solution in the Notebook.

## Question 2 [3 marks]

We will use the file "TaleOfTwoCities.txt" (available in Notebook 2, and also [here](./files/TaleOfTwoCities.txt)). Please build a pipeline to do the following 

1. Read the file in
2. Tokenize it
3. Remove stop words
4. Map all words to lowercase
5. Find the 15 most frequent words and the 15 least frequent words. Show them (30 words in total).

The order of the above steps can be different but all should be included (and give the same results).

Provide your solution in the Notebook.


In [48]:
val sparkVersion = "2.0.1"
val scalaVersion = scala.util.Properties.versionNumberString

sparkVersion: String = "2.0.1"
scalaVersion: String = "2.11.8"

In [49]:
classpath.add(
    "org.apache.spark" %% "spark-yarn" % sparkVersion,
    "org.apache.spark" %% "spark-mllib" % sparkVersion
)

0 new artifact(s)


In [50]:

import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions.col
import org.apache.spark.ml.feature.Tokenizer
import org.apache.spark.sql.SparkSession._
import org.apache.spark.sql.functions._
import org.apache.spark.ml.feature.NGram
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.feature.StopWordsRemover
import org.apache.spark.ml.{Pipeline, PipelineModel}

import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions.col
import org.apache.spark.ml.feature.Tokenizer
import org.apache.spark.sql.SparkSession._
import org.apache.spark.sql.functions._
import org.apache.spark.ml.feature.NGram
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.feature.StopWordsRemover
import org.apache.spark.ml.{Pipeline, PipelineModel}

In [51]:
//Exercise 1
import scala.io.Source


object Masters {
  def main(args: Array[String]) {
      val sparkSession = SparkSession.builder
        .master("local")
        .appName("Exercise_1")
        .getOrCreate()

    val sc = sparkSession.sparkContext

    val df = sparkSession.read
            .format("com.databricks.spark.csv")
            .option("header", "false") //reading the headers
            .option("mode", "DROPMALFORMED")
            .load("files/CensusIncomeData.csv");
            
      val selection = df.select("_c3")
      
      //df.show()

      //val counting_occurances = df.groupBy("_c3").count().show()
      
      
     val masters = df.filter(col("_c3").like("%Masters"))
     val masters_women = masters.filter(col("_c9") === " Female").count()
    
    
     println(s"The number of Women that have a Masters Degree is: $masters_women")
    
   
   }
}
Masters.main(Array())

The number of Women that have a Masters Degree is: 420


import scala.io.Source
defined object Masters

In [52]:
// Exercise 2

object tales {
    def main() {
        val sparkSession = SparkSession.builder
            .master("local")
            .appName("Exercise_2")
            .getOrCreate()
        
        val sc = sparkSession.sparkContext  
        
        import sparkSession.implicits._

        val tales = sparkSession.read.text("files/TaleOfTwoCities.txt").as[String]
        //tales.show()
        
        val tokenizer = new Tokenizer().setInputCol("value").setOutputCol("tokens")
        val stop_word_remove = new StopWordsRemover().setInputCol(tokenizer.getOutputCol).setOutputCol("extracted_words")
        
        val pipeline = new Pipeline().setStages(Array(tokenizer, stop_word_remove))
        val model = pipeline.fit(tales)
        val results = model.transform(tales)
        
        
        val top = results.withColumn("extracted_words", explode(col("extracted_words")))
        val most = top.groupBy("extracted_words").agg(count("*") as "most_frequent").orderBy(desc("most_frequent")).show(15)
        
        val last = results.withColumn("extracted_words", explode(col("extracted_words")))
        val least = last.groupBy("extracted_words").agg(count("*") as "least_frequent").orderBy("least_frequent").show(15)
        
    }
}

tales.main()

+---------------+-------------+
|extracted_words|most_frequent|
+---------------+-------------+
|               |           17|
|            one|            5|
|           year|            4|
|           shot|            4|
|          seven|            4|
|          them,|            4|
|       thousand|            4|
|          large|            3|
|        hundred|            3|
|           now,|            3|
|          fired|            3|
|         light,|            2|
|        burning|            2|
|            jaw|            2|
|         direct|            2|
+---------------+-------------+
only showing top 15 rows

+---------------+--------------+
|extracted_words|least_frequent|
+---------------+--------------+
|        present|             1|
|       conceded|             1|
|         worked|             1|
|        snipped|             1|
|       received|             1|
|       spending|             1|
|      character|             1|
|         taking|             1|
|  

defined object tales